In [56]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
import scipy.stats as st
import statsmodels.api as sm
import numpy as np

from VaR_func import *
from tests import *

In [57]:
def test_exceeds_vector(vector, alpha=.95):
    return pd.DataFrame({'empiryczne': [vector.mean(),
                                        vector.var(),
                                        stats.binomtest(int(vector.sum()), n=vector.size, p=1-alpha).pvalue,
                                        kupiec(vector, alpha),
                                        christoffersen(vector)],
                         'teoretyczne': [1 - alpha,
                                         alpha*(1-alpha),
                                         '-',
                                         '-',
                                         '-']},
                        index=['średnia', 'wariancja',
                               'binomtest p-wartość',
                               'Kupiec - p-wartość',
                               'christoffersen p-wartość'])

In [42]:
data = pd.read_csv(".\\data\\ropa.csv", index_col='Date', header=0)
data['returns'] = data.interpolate(type='linear').pct_change()
data.index = pd.to_datetime(data.index)
r = list(data['returns'])
r = r[1:]

dist_type = "genhyperbolic"
filename = './data_distfit/fitted_{}.csv'.format(dist_type)
df1 = pd.read_csv(filename, index_col=0)
df1

,t_min,t_max,params,SSE,ecdf_quantiles,VaR_95,VaR_99,EVaR_95,EVaR_99
-1,0,798,"(-1.3116152611561311, 1.8686302875552796, -0.5...",0.044717,[-1.30628881e-01 -1.21261119e-01 -8.23572838e-...,0.037237,0.056042,0.048969,0.067759
0,0,499,"(1.885124886138522, 0.1750444811464994, -0.030...",0.021262,"[-0.1306288805140844, -0.12126111859385258, -0...",0.040505,0.061625,0.053593,0.074156
1,1,500,"(1.8696260201581674, 0.030163851060952762, -0....",0.023045,"[-0.1306288805140844, -0.12126111859385258, -0...",0.040372,0.061290,0.053335,0.073707
2,2,501,"(1.9166427999093651, 0.10111038610401962, -0.0...",0.020939,"[-0.1306288805140844, -0.12126111859385258, -0...",0.040378,0.061329,0.053361,0.073745
3,3,502,"(1.8274890497976828, 0.001777716117093763, -0....",0.023313,"[-0.1306288805140844, -0.12126111859385258, -0...",0.040189,0.061252,0.053244,0.073779
...,...,...,...,...,...,...,...,...,...
293,293,792,"(6.369557838896252, 0.36362576689090265, -0.08...",0.047251,"[-0.08235728377794271, -0.07925833692240081, -...",0.034775,0.049088,0.043600,0.056757
294,294,793,"(5.652127691467884, 0.0020846996881396275, -0....",0.048553,"[-0.08235728377794271, -0.07925833692240081, -...",0.034576,0.049101,0.043536,0.056968
295,295,794,"(5.857908487208633, 0.36306422412408157, -0.08...",0.049664,"[-0.08235728377794271, -0.07925833692240081, -...",0.034553,0.048944,0.043429,0.056709
296,296,795,"(5.921132568214928, 0.07325839213392335, -0.01...",0.048255,"[-0.08235728377794271, -0.07925833692240081, -...",0.034596,0.048990,0.043474,0.056751


In [43]:
v95 = df1['VaR_95'][1:]
v99 = df1['VaR_99'][1:]

In [44]:
r_test = r[-298:]
len(r_test)

298

In [45]:
okna = df1['ecdf_quantiles'][1:]
okna = [eval(i) for i in okna]
len(okna)

298

In [46]:
for i in range(len(r_test)):
    print(r_test[i] in okna[i])

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False

In [47]:
I_95 = []
I_99 = []
for i in range(len(r_test)):
    I_95.append(v95[i] < r_test[i])
    I_99.append(v99[i] < r_test[i])

In [48]:
kupiec(np.array(I_95), .95)

0.007447710138538399

In [61]:
kupiec(np.array(I_99), .95)

1.5630223488827255e-06

In [50]:
christoffersen(np.array(I_95))

0.6492634807020252

In [51]:
scoring_quantile(r_test, v95)

0      0.001408
1      0.002438
2      0.001114
3      0.001594
4      0.001696
         ...   
293    0.002069
294    0.001270
295    0.001560
296    0.002457
297    0.002154
Name: VaR_95, Length: 298, dtype: float64

In [52]:
scoring_quantile(r_test, v99)

0      0.002464
1      0.003484
2      0.002161
3      0.002647
4      0.002745
         ...   
293    0.002784
294    0.001996
295    0.002280
296    0.003177
297    0.002878
Name: VaR_99, Length: 298, dtype: float64

In [62]:
test_exceeds_vector(np.array(I_99), alpha=0.99)

,empiryczne,teoretyczne
średnia,0.003356,0.01
wariancja,0.003344,0.0099
binomtest p-wartość,0.381180,-
Kupiec - p-wartość,0.180998,-
christoffersen p-wartość,0.994624,-
